## Spark SQL

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

#### RDD VS Data Frame

##### RDD

In [0]:
dataRDD = sc.parallelize([("Brooke", 20), ("Denny", 31), ("Jules", 30),
("TD", 35), ("Brooke", 25)])

agesRDD = (dataRDD.map(lambda x: (x[0], (x[1], 1))) 
           .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) 
           .map(lambda x: (x[0], x[1][0]/x[1][1])))
agesRDD.collect()

Out[71]: [('Brooke', 22.5), ('Denny', 31.0), ('TD', 35.0), ('Jules', 30.0)]

##### DataFrame

In [0]:
data_df = spark.createDataFrame([("Brooke", 20), ("Denny", 31), ("Jules", 30),("TD", 35), ("Brooke", 25)], ["name", "age"])
avg_df = data_df.groupBy("name").agg(avg("age"))
avg_df.show()

+------+--------+
 name|avg(age)|
+------+--------+
Brooke| 22.5|
 Denny| 31.0|
 Jules| 30.0|
 TD| 35.0|
+------+--------+

### Creating DataFrame

In [0]:
data = [[1, "Jules", "Damji", "https://tinyurl.1", "1/4/2016", 4535, ["twitter", "LinkedIn"]],
           [2, "Brooke","Wenig", "https://tinyurl.2", "5/5/2018", 8908, ["twitter",
    "LinkedIn"]],
           [3, "Denny", "Lee", "https://tinyurl.3", "6/7/2019", 7659, ["web",
    "twitter", "FB", "LinkedIn"]],
           [4, "Tathagata", "Das", "https://tinyurl.4", "5/12/2018", 10568,
    ["twitter", "FB"]],
           [5, "Matei","Zaharia", "https://tinyurl.5", "5/14/2014", 40578, ["web",
    "twitter", "FB", "LinkedIn"]],
           [6, "Reynold", "Xin", "https://tinyurl.6", "3/2/2015", 25568,
    ["twitter", "LinkedIn"]]
          ]



#### Defining a schema

##### Using DDL

In [0]:
schema = "`Id` INT, `First` STRING, `Last` STRING, `Url` STRING, `Published` STRING, `Hits` INT, `Campaigns` ARRAY<STRING>"

In [0]:
blogs_df = spark.createDataFrame(data, schema)
blogs_df.show()
blogs_df.printSchema()

+---+---------+-------+-----------------+---------+-----+--------------------+
 Id| First| Last| Url|Published| Hits| Campaigns|
+---+---------+-------+-----------------+---------+-----+--------------------+
 1| Jules| Damji|https://tinyurl.1| 1/4/2016| 4535| [twitter, LinkedIn]|
 2| Brooke| Wenig|https://tinyurl.2| 5/5/2018| 8908| [twitter, LinkedIn]|
 3| Denny| Lee|https://tinyurl.3| 6/7/2019| 7659|[web, twitter, FB...|
 4|Tathagata| Das|https://tinyurl.4|5/12/2018|10568| [twitter, FB]|
 5| Matei|Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB...|
 6| Reynold| Xin|https://tinyurl.6| 3/2/2015|25568| [twitter, LinkedIn]|
+---+---------+-------+-----------------+---------+-----+--------------------+

root
-- Id: integer (nullable = true)
-- First: string (nullable = true)
-- Last: string (nullable = true)
-- Url: string (nullable = true)
-- Published: string (nullable = true)
-- Hits: integer (nullable = true)
-- Campaigns: array (nullable = true)
 |-- element: string (containsNull = true)

##### Schema Created Programmatically

In [0]:

schema2 = StructType([
  StructField("Id", IntegerType(), False),
  StructField("First", StringType(), False),
  StructField("Last", StringType(), False),
  StructField("Url", StringType(), True),
  StructField("Published", StringType(), False),
  StructField("Hits", IntegerType(), False),
  StructField("Campaigns", ArrayType(StringType(), True), False)
])

In [0]:
blogs_df = spark.createDataFrame(data, schema2)
blogs_df.show()
blogs_df.printSchema()

+---+---------+-------+-----------------+---------+-----+--------------------+
 Id| First| Last| Url|Published| Hits| Campaigns|
+---+---------+-------+-----------------+---------+-----+--------------------+
 1| Jules| Damji|https://tinyurl.1| 1/4/2016| 4535| [twitter, LinkedIn]|
 2| Brooke| Wenig|https://tinyurl.2| 5/5/2018| 8908| [twitter, LinkedIn]|
 3| Denny| Lee|https://tinyurl.3| 6/7/2019| 7659|[web, twitter, FB...|
 4|Tathagata| Das|https://tinyurl.4|5/12/2018|10568| [twitter, FB]|
 5| Matei|Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB...|
 6| Reynold| Xin|https://tinyurl.6| 3/2/2015|25568| [twitter, LinkedIn]|
+---+---------+-------+-----------------+---------+-----+--------------------+

root
-- Id: integer (nullable = false)
-- First: string (nullable = false)
-- Last: string (nullable = false)
-- Url: string (nullable = true)
-- Published: string (nullable = false)
-- Hits: integer (nullable = false)
-- Campaigns: array (nullable = false)
 |-- element: string (containsNull = true)

In [0]:
blogs_df.schema

Out[78]: StructType(List(StructField(Id,IntegerType,false),StructField(First,StringType,false),StructField(Last,StringType,false),StructField(Url,StringType,true),StructField(Published,StringType,false),StructField(Hits,IntegerType,false),StructField(Campaigns,ArrayType(StringType,true),false)))

#### Creating DataFranes from Data Sources

* csv
* json
* text
* jdbc
* orc
* parquet

##### Create a DataFrame from JSON file

In [0]:
people_df = spark.read.json("dbfs:/databricks-datasets/samples/people/people.json")
people_df.show()
people_df.printSchema()

+---+------+
age| name|
+---+------+
 40| Jane|
 30| Andy|
 50|Justin|
+---+------+

root
-- age: long (nullable = true)
-- name: string (nullable = true)

##### Create a DataFrame from the NYT COVID data CSV file

In [0]:
covid_df = spark.read.csv("/databricks-datasets/COVID/covid-19-data/us-counties.csv")
covid_df.show()
covid_df.printSchema()

+----------+-----------+----------+-----+-----+------+
 _c0| _c1| _c2| _c3| _c4| _c5|
+----------+-----------+----------+-----+-----+------+
 date| county| state| fips|cases|deaths|
2020-01-21| Snohomish|Washington|53061| 1| 0|
2020-01-22| Snohomish|Washington|53061| 1| 0|
2020-01-23| Snohomish|Washington|53061| 1| 0|
2020-01-24| Cook| Illinois|17031| 1| 0|
2020-01-24| Snohomish|Washington|53061| 1| 0|
2020-01-25| Orange|California|06059| 1| 0|
2020-01-25| Cook| Illinois|17031| 1| 0|
2020-01-25| Snohomish|Washington|53061| 1| 0|
2020-01-26| Maricopa| Arizona|04013| 1| 0|
2020-01-26|Los Angeles|California|06037| 1| 0|
2020-01-26| Orange|California|06059| 1| 0|
2020-01-26| Cook| Illinois|17031| 1| 0|
2020-01-26| Snohomish|Washington|53061| 1| 0|
2020-01-27| Maricopa| Arizona|04013| 1| 0|
2020-01-27|Los Angeles|California|06037| 1| 0|
2020-01-27| Orange|California|06059| 1| 0|
2020-01-27| Cook| Illinois|17031| 1| 0|
2020-01-27| Snohomish|Washington|53061| 1| 0|
2020-01-28| Maricopa| Arizona|04013| 1| 0|
+----------+-----------+----------+-----+-----+------+
only showing top 20 rows

root
-- _c0: string (nullable = true)
-- _c1: string (nullable = true)
-- _c2: string (nullable = true)
-- _c3: string (nullable = true)
-- _c4: string (nullable = true)
-- _c5: string (nullable = true)

[Spark Documentation](https://spark.apache.org/docs/latest/)

In [0]:
covid_df_ = spark.read.csv("/databricks-datasets/COVID/covid-19-data/us-counties.csv", header=True, inferSchema=True)
covid_df_.show()

+----------+-----------+----------+-----+-----+------+
 date| county| state| fips|cases|deaths|
+----------+-----------+----------+-----+-----+------+
2020-01-21| Snohomish|Washington|53061| 1| 0|
2020-01-22| Snohomish|Washington|53061| 1| 0|
2020-01-23| Snohomish|Washington|53061| 1| 0|
2020-01-24| Cook| Illinois|17031| 1| 0|
2020-01-24| Snohomish|Washington|53061| 1| 0|
2020-01-25| Orange|California| 6059| 1| 0|
2020-01-25| Cook| Illinois|17031| 1| 0|
2020-01-25| Snohomish|Washington|53061| 1| 0|
2020-01-26| Maricopa| Arizona| 4013| 1| 0|
2020-01-26|Los Angeles|California| 6037| 1| 0|
2020-01-26| Orange|California| 6059| 1| 0|
2020-01-26| Cook| Illinois|17031| 1| 0|
2020-01-26| Snohomish|Washington|53061| 1| 0|
2020-01-27| Maricopa| Arizona| 4013| 1| 0|
2020-01-27|Los Angeles|California| 6037| 1| 0|
2020-01-27| Orange|California| 6059| 1| 0|
2020-01-27| Cook| Illinois|17031| 1| 0|
2020-01-27| Snohomish|Washington|53061| 1| 0|
2020-01-28| Maricopa| Arizona| 4013| 1| 0|
2020-01-28|Los Angeles|California| 6037| 1| 0|
+----------+-----------+----------+-----+-----+------+
only showing top 20 rows

In [0]:
covid_df_.printSchema()

root
-- date: string (nullable = true)
-- county: string (nullable = true)
-- state: string (nullable = true)
-- fips: integer (nullable = true)
-- cases: integer (nullable = true)
-- deaths: integer (nullable = true)

In [0]:
covid_schema = StructType([
  StructField("Date",DateType(),True),
  StructField("county",StringType(),True),
  StructField("state",StringType(),True),
  StructField("fips",IntegerType(),True),
  StructField("cases",IntegerType(),True),
  StructField("deaths",IntegerType(),True)
])

In [0]:
covid_df = spark.read.schema(covid_schema).csv("/databricks-datasets/COVID/covid-19-data/us-counties.csv", header=True)
covid_df.show()

+----------+-----------+----------+-----+-----+------+
 Date| county| state| fips|cases|deaths|
+----------+-----------+----------+-----+-----+------+
2020-01-21| Snohomish|Washington|53061| 1| 0|
2020-01-22| Snohomish|Washington|53061| 1| 0|
2020-01-23| Snohomish|Washington|53061| 1| 0|
2020-01-24| Cook| Illinois|17031| 1| 0|
2020-01-24| Snohomish|Washington|53061| 1| 0|
2020-01-25| Orange|California| 6059| 1| 0|
2020-01-25| Cook| Illinois|17031| 1| 0|
2020-01-25| Snohomish|Washington|53061| 1| 0|
2020-01-26| Maricopa| Arizona| 4013| 1| 0|
2020-01-26|Los Angeles|California| 6037| 1| 0|
2020-01-26| Orange|California| 6059| 1| 0|
2020-01-26| Cook| Illinois|17031| 1| 0|
2020-01-26| Snohomish|Washington|53061| 1| 0|
2020-01-27| Maricopa| Arizona| 4013| 1| 0|
2020-01-27|Los Angeles|California| 6037| 1| 0|
2020-01-27| Orange|California| 6059| 1| 0|
2020-01-27| Cook| Illinois|17031| 1| 0|
2020-01-27| Snohomish|Washington|53061| 1| 0|
2020-01-28| Maricopa| Arizona| 4013| 1| 0|
2020-01-28|Los Angeles|California| 6037| 1| 0|
+----------+-----------+----------+-----+-----+------+
only showing top 20 rows

In [0]:
covid_df.printSchema()

root
-- Date: date (nullable = true)
-- county: string (nullable = true)
-- state: string (nullable = true)
-- fips: integer (nullable = true)
-- cases: integer (nullable = true)
-- deaths: integer (nullable = true)

In [0]:
patients_df = spark.read.csv("dbfs:/databricks-datasets/rwe/ehr/csv/patients.csv", header=True, inferSchema=True)
patients_df.show()

+--------------------+----------+----------+-----------+---------+----------+------+-------+----------+------+-----------+-------+--------+---------------+------+--------------------+--------------------+---------------+-------------+----+
 Id| BIRTHDATE| DEATHDATE| SSN| DRIVERS| PASSPORT|PREFIX| FIRST| LAST|SUFFIX| MAIDEN|MARITAL| RACE| ETHNICITY|GENDER| BIRTHPLACE| ADDRESS| CITY| STATE| ZIP|
+--------------------+----------+----------+-----------+---------+----------+------+-------+----------+------+-----------+-------+--------+---------------+------+--------------------+--------------------+---------------+-------------+----+
a45a0f84-1d16-451...|1979-04-05| null|999-95-4871|S99995591| X718276X| Mr.|Gilbert| Medhurst| null| null| M| white| american| M|West Tisbury Mas...|696 Schmeler Stra...| Leominster|Massachusetts|1420|
cc3e87e4-6172-45c...|1979-04-01| null|999-56-1714|S99954952|X26053750X| Mr.| Corey| Hoeger| null| null| M| white| english| M|East Bridgewater ...|1080 Klein Highla...| Newton|Massachusetts|2458|
4edfaae9-eef0-430...|1948-01-09| null|999-65-8945|S99996643|X78731815X| Mrs.| Brook| Dooley| null|Greenfelder| M| white|french_canadian| F|Springfield Mass...|253 Gutkowski App...| Belmont|Massachusetts|2138|
7ff0403d-6cc4-48a...|1956-01-12| null|999-67-6216|S99945217|X30828362X| Mr.|Norbert| Corkery| null| null| M| white| german| M|Pembroke Massach...| 520 White Bay|Barnstable Town|Massachusetts|2601|
2f905f97-cf1f-4f0...|1998-05-30| null|999-28-1698|S99990095| X1436582X| Ms.| Maisie|McCullough| null| null| null| white|french_canadian| F|Worcester Massac...| 118 Marks Port| Lynn|Massachusetts|1901|
b9b551cf-adf6-4f2...|1970-09-05|2014-03-15|999-19-5833|S99996196|X26794549X| Mr.| Monte|Macejkovic| null| null| M| black| dominican| M|Brookline Massac...|995 Heller Rest A...| Springfield|Massachusetts|1013|
e747f9b3-480b-4ce...|1994-07-07| null|999-75-6843|S99958855|X20317878X| Ms.|Lucinda| Bayer| null| null| null| white| irish| F|Melrose Massachu...|338 Lebsack Glen ...| Ware|Massachusetts|1082|
61bb8fc6-b6ce-472...|1990-09-28| null|999-60-2463|S99916176|X57174560X| Mrs.| Carla| Badillo| null| Ontiveros| M|hispanic| mexican| F|Mexico City Mexi...|952 Farrell Boule...| Peabody|Massachusetts|1940|
13e5d9d1-9756-4d5...|2011-09-20| null|999-79-5029| null| null| null|Analisa|Morissette| null| null| null| white| irish| F|Templeton Massac...| 949 Turner Lane| Middleborough|Massachusetts|2346|
3919e818-9080-4c1...|1998-02-09| null|999-28-8651|S99915632|X72780201X| Mr.| Quincy| Lebsack| null| null| null| white| french| M|Townsend Massach...| 452 Brakus Gardens| Chelsea|Massachusetts|2149|
fdd3cddc-7b4a-468...|1988-10-01| null|999-11-6581|S99953793| X5626852X| Mr.| Errol| Tillman| null| null| M| white| polish| M|Tewksbury Massac...|783 Walsh Ranch A...| Somerville|Massachusetts|2138|
26aaef22-2840-4ce...|1986-02-26| null|999-58-6994|S99952390| X4942798X| Mr.| Jerald| Waters| null| null| S| white| swedish| M|Swansea Massachu...|413 Boyle Station...| Chelmsford|Massachusetts|1824|
d6b2eece-060d-41d...|2015-03-01| null|999-12-1401| null| null| null| Kip| Mitchell| null| null| null| white| french| M|Bellingham Massa...| 606 Strosin Park| Boston|Massachusetts|2108|
e958b972-c33c-455...|1975-04-24| null|999-17-4292|S99947722|X55742300X| Mr.| Pablo| Rivero| null| null| M| black| west_indian| M|Santiago de los C...|555 Langworth Pat...| Ayer|Massachusetts|1432|
9bbe5f47-dbac-4ff...|1970-09-05| null|999-20-6466|S99963920|X21871050X| Mr.| Judson| Franecki| null| null| S| black| dominican| M|Hanson Massachus...| 759 Hickle Fort| Springfield|Massachusetts|1013|
4b575928-56c0-4cf...|1949-01-03| null|999-41-2337|S99933266|X32569297X| Mr.| Stuart| Fadel| null| null| S| asian| asian_indian| M|Springfield Mass...|813 Ebert Union A...| Boston|Massachusetts|2108|
079a81dd-9e6b-4ce...|1991-02-06| null|999-68-4829|S99926906|X51211096X| Mr.| Bryon| Daugherty| null| null| M| native|american_indian| M|Quincy Massachus...|496 Kertzmann Cam...| Winthr

### DataFrames Operations

#### Transformations

* select
* filter 
* where
* distinct
* dropDuplicates
* sort
* orderBy
* limit
* union
* withColumn
* withColumnRenamed
* drop
* sample
* join
* groupBy
*

#### Action

* show([numRows], [[truncate]])
* head([n])
* first([n])
* take(n)
* takeAsList(n)
* collect()
* conut()

#### Aggregation Function

* count(col)
* countDistinct(col)
* min(col)
* max(col)
* sum(col)
* sumDistinct(col)
* avg(col)

### Working with DataFrames

##### Show our data

###### Covid DataFrame

In [0]:
covid_df.head()

Out[87]: Row(Date=datetime.date(2020, 1, 21), county='Snohomish', state='Washington', fips=53061, cases=1, deaths=0)

In [0]:
covid_df.show()

+----------+-----------+----------+-----+-----+------+
 Date| county| state| fips|cases|deaths|
+----------+-----------+----------+-----+-----+------+
2020-01-21| Snohomish|Washington|53061| 1| 0|
2020-01-22| Snohomish|Washington|53061| 1| 0|
2020-01-23| Snohomish|Washington|53061| 1| 0|
2020-01-24| Cook| Illinois|17031| 1| 0|
2020-01-24| Snohomish|Washington|53061| 1| 0|
2020-01-25| Orange|California| 6059| 1| 0|
2020-01-25| Cook| Illinois|17031| 1| 0|
2020-01-25| Snohomish|Washington|53061| 1| 0|
2020-01-26| Maricopa| Arizona| 4013| 1| 0|
2020-01-26|Los Angeles|California| 6037| 1| 0|
2020-01-26| Orange|California| 6059| 1| 0|
2020-01-26| Cook| Illinois|17031| 1| 0|
2020-01-26| Snohomish|Washington|53061| 1| 0|
2020-01-27| Maricopa| Arizona| 4013| 1| 0|
2020-01-27|Los Angeles|California| 6037| 1| 0|
2020-01-27| Orange|California| 6059| 1| 0|
2020-01-27| Cook| Illinois|17031| 1| 0|
2020-01-27| Snohomish|Washington|53061| 1| 0|
2020-01-28| Maricopa| Arizona| 4013| 1| 0|
2020-01-28|Los Angeles|California| 6037| 1| 0|
+----------+-----------+----------+-----+-----+------+
only showing top 20 rows

###### PatientsDataFrame

In [0]:
patients_df.take(1)

Out[89]: [Row(Id='a45a0f84-1d16-4517-9aa8-271c59f339fd', BIRTHDATE='1979-04-05', DEATHDATE=None, SSN='999-95-4871', DRIVERS='S99995591', PASSPORT='X718276X', PREFIX='Mr.', FIRST='Gilbert', LAST='Medhurst', SUFFIX=None, MAIDEN=None, MARITAL='M', RACE='white', ETHNICITY='american', GENDER='M', BIRTHPLACE='West Tisbury Massachusetts US', ADDRESS='696 Schmeler Stravenue Apt 45', CITY='Leominster', STATE='Massachusetts', ZIP=1420)]

In [0]:
patients_df.show()

+--------------------+----------+----------+-----------+---------+----------+------+-------+----------+------+-----------+-------+--------+---------------+------+--------------------+--------------------+---------------+-------------+----+
 Id| BIRTHDATE| DEATHDATE| SSN| DRIVERS| PASSPORT|PREFIX| FIRST| LAST|SUFFIX| MAIDEN|MARITAL| RACE| ETHNICITY|GENDER| BIRTHPLACE| ADDRESS| CITY| STATE| ZIP|
+--------------------+----------+----------+-----------+---------+----------+------+-------+----------+------+-----------+-------+--------+---------------+------+--------------------+--------------------+---------------+-------------+----+
a45a0f84-1d16-451...|1979-04-05| null|999-95-4871|S99995591| X718276X| Mr.|Gilbert| Medhurst| null| null| M| white| american| M|West Tisbury Mas...|696 Schmeler Stra...| Leominster|Massachusetts|1420|
cc3e87e4-6172-45c...|1979-04-01| null|999-56-1714|S99954952|X26053750X| Mr.| Corey| Hoeger| null| null| M| white| english| M|East Bridgewater ...|1080 Klein Highla...| Newton|Massachusetts|2458|
4edfaae9-eef0-430...|1948-01-09| null|999-65-8945|S99996643|X78731815X| Mrs.| Brook| Dooley| null|Greenfelder| M| white|french_canadian| F|Springfield Mass...|253 Gutkowski App...| Belmont|Massachusetts|2138|
7ff0403d-6cc4-48a...|1956-01-12| null|999-67-6216|S99945217|X30828362X| Mr.|Norbert| Corkery| null| null| M| white| german| M|Pembroke Massach...| 520 White Bay|Barnstable Town|Massachusetts|2601|
2f905f97-cf1f-4f0...|1998-05-30| null|999-28-1698|S99990095| X1436582X| Ms.| Maisie|McCullough| null| null| null| white|french_canadian| F|Worcester Massac...| 118 Marks Port| Lynn|Massachusetts|1901|
b9b551cf-adf6-4f2...|1970-09-05|2014-03-15|999-19-5833|S99996196|X26794549X| Mr.| Monte|Macejkovic| null| null| M| black| dominican| M|Brookline Massac...|995 Heller Rest A...| Springfield|Massachusetts|1013|
e747f9b3-480b-4ce...|1994-07-07| null|999-75-6843|S99958855|X20317878X| Ms.|Lucinda| Bayer| null| null| null| white| irish| F|Melrose Massachu...|338 Lebsack Glen ...| Ware|Massachusetts|1082|
61bb8fc6-b6ce-472...|1990-09-28| null|999-60-2463|S99916176|X57174560X| Mrs.| Carla| Badillo| null| Ontiveros| M|hispanic| mexican| F|Mexico City Mexi...|952 Farrell Boule...| Peabody|Massachusetts|1940|
13e5d9d1-9756-4d5...|2011-09-20| null|999-79-5029| null| null| null|Analisa|Morissette| null| null| null| white| irish| F|Templeton Massac...| 949 Turner Lane| Middleborough|Massachusetts|2346|
3919e818-9080-4c1...|1998-02-09| null|999-28-8651|S99915632|X72780201X| Mr.| Quincy| Lebsack| null| null| null| white| french| M|Townsend Massach...| 452 Brakus Gardens| Chelsea|Massachusetts|2149|
fdd3cddc-7b4a-468...|1988-10-01| null|999-11-6581|S99953793| X5626852X| Mr.| Errol| Tillman| null| null| M| white| polish| M|Tewksbury Massac...|783 Walsh Ranch A...| Somerville|Massachusetts|2138|
26aaef22-2840-4ce...|1986-02-26| null|999-58-6994|S99952390| X4942798X| Mr.| Jerald| Waters| null| null| S| white| swedish| M|Swansea Massachu...|413 Boyle Station...| Chelmsford|Massachusetts|1824|
d6b2eece-060d-41d...|2015-03-01| null|999-12-1401| null| null| null| Kip| Mitchell| null| null| null| white| french| M|Bellingham Massa...| 606 Strosin Park| Boston|Massachusetts|2108|
e958b972-c33c-455...|1975-04-24| null|999-17-4292|S99947722|X55742300X| Mr.| Pablo| Rivero| null| null| M| black| west_indian| M|Santiago de los C...|555 Langworth Pat...| Ayer|Massachusetts|1432|
9bbe5f47-dbac-4ff...|1970-09-05| null|999-20-6466|S99963920|X21871050X| Mr.| Judson| Franecki| null| null| S| black| dominican| M|Hanson Massachus...| 759 Hickle Fort| Springfield|Massachusetts|1013|
4b575928-56c0-4cf...|1949-01-03| null|999-41-2337|S99933266|X32569297X| Mr.| Stuart| Fadel| null| null| S| asian| asian_indian| M|Springfield Mass...|813 Ebert Union A...| Boston|Massachusetts|2108|
079a81dd-9e6b-4ce...|1991-02-06| null|999-68-4829|S99926906|X51211096X| Mr.| Bryon| Daugherty| null| null| M| native|american_indian| M|Quincy Massachus...|496 Kertzmann Cam...| Winthr

##### Counting number of rows

In [0]:
covid_df.count()

Out[91]: 917272

In [0]:
patients_df.count()

Out[92]: 11737

##### Simple Select

In [0]:
covid_df.select(col("county"), col("cases")).show(5)
covid_df.select(covid_df["county"], covid_df["cases"]).show(5)
covid_df.select("county", "cases").show(5)

+---------+-----+
 county|cases|
+---------+-----+
Snohomish| 1|
Snohomish| 1|
Snohomish| 1|
 Cook| 1|
Snohomish| 1|
+---------+-----+
only showing top 5 rows

+---------+-----+
 county|cases|
+---------+-----+
Snohomish| 1|
Snohomish| 1|
Snohomish| 1|
 Cook| 1|
Snohomish| 1|
+---------+-----+
only showing top 5 rows

+---------+-----+
 county|cases|
+---------+-----+
Snohomish| 1|
Snohomish| 1|
Snohomish| 1|
 Cook| 1|
Snohomish| 1|
+---------+-----+
only showing top 5 rows

In [0]:
covid_df.select(year("Date").alias("Year"), col("cases") + 1).show(5)

+----+-----------+
Year|(cases + 1)|
+----+-----------+
2020| 2|
2020| 2|
2020| 2|
2020| 2|
2020| 2|
+----+-----------+
only showing top 5 rows

##### Get the latest informations

In [0]:
#covid_df.select("Date", "county", "cases", "deaths").orderBy(covid_df["Date"].desc()).show()
covid_df.select("Date", "county", "cases", "deaths").sort(covid_df["Date"].desc()).show()

+----------+--------+-----+------+
 Date| county|cases|deaths|
+----------+--------+-----+------+
2021-01-10| Autauga| 4879| 54|
2021-01-10|Crenshaw| 1123| 38|
2021-01-10| Baldwin|15327| 173|
2021-01-10| Barbour| 1658| 35|
2021-01-10| Bibb| 2051| 48|
2021-01-10| Blount| 5066| 77|
2021-01-10| Bullock| 953| 28|
2021-01-10| Butler| 1632| 57|
2021-01-10| Calhoun|10497| 178|
2021-01-10|Chambers| 2639| 66|
2021-01-10|Cherokee| 1537| 24|
2021-01-10| Chilton| 3253| 54|
2021-01-10| Choctaw| 501| 22|
2021-01-10| Clarke| 2633| 26|
2021-01-10| Clay| 1230| 35|
2021-01-10|Cleburne| 1118| 18|
2021-01-10| Coffee| 4145| 43|
2021-01-10| Colbert| 5081| 61|
2021-01-10| Conecuh| 907| 18|
2021-01-10| Coosa| 579| 9|
+----------+--------+-----+------+
only showing top 20 rows

##### Get the latest informations from Los Angeles

In [0]:
covid_df.sort(covid_df["Date"].desc()).filter(covid_df["county"] == "Los Angeles").show()
#covid_df.sort(covid_df["Date"].desc()).where(covid_df["county"] == "Los Angeles").show()

+----------+-----------+----------+----+------+------+
 Date| county| state|fips| cases|deaths|
+----------+-----------+----------+----+------+------+
2021-01-10|Los Angeles|California|6037|920560| 12250|
2021-01-09|Los Angeles|California|6037|906554| 12084|
2021-01-08|Los Angeles|California|6037|889787| 11863|
2021-01-07|Los Angeles|California|6037|871749| 11545|
2021-01-06|Los Angeles|California|6037|852510| 11328|
2021-01-05|Los Angeles|California|6037|840956| 11071|
2021-01-04|Los Angeles|California|6037|827843| 10850|
2021-01-03|Los Angeles|California|6037|818959| 10773|
2021-01-02|Los Angeles|California|6037|806523| 10682|
2021-01-01|Los Angeles|California|6037|790895| 10552|
2020-12-31|Los Angeles|California|6037|770915| 10345|
2020-12-30|Los Angeles|California|6037|756412| 10056|
2020-12-29|Los Angeles|California|6037|746381| 9782|
2020-12-28|Los Angeles|California|6037|733612| 9555|
2020-12-27|Los Angeles|California|6037|720099| 9482|
2020-12-26|Los Angeles|California|6037|706712| 9438|
2020-12-25|Los Angeles|California|6037|677538| 9299|
2020-12-24|Los Angeles|California|6037|677538| 9299|
2020-12-23|Los Angeles|California|6037|664193| 9153|
2020-12-22|Los Angeles|California|6037|647766| 9016|
+----------+-----------+----------+----+------+------+
only showing top 20 rows

##### Get informations about counities in the US

In [0]:
covid_df.select("county").distinct().count()
#covid_df.select("county", "state").dropDuplicates(["county"]).show()


Out[97]: 1930

##### Compute number of heald people

In [0]:
covid_df_with_heald = covid_df.withColumn("heald", (col('cases')- col('deaths')))
covid_df_with_heald.sort(covid_df_with_heald["Date"].desc()).show()

+----------+--------+-------+----+-----+------+-----+
 Date| county| state|fips|cases|deaths|heald|
+----------+--------+-------+----+-----+------+-----+
2021-01-10| Autauga|Alabama|1001| 4879| 54| 4825|
2021-01-10|Crenshaw|Alabama|1041| 1123| 38| 1085|
2021-01-10| Baldwin|Alabama|1003|15327| 173|15154|
2021-01-10| Barbour|Alabama|1005| 1658| 35| 1623|
2021-01-10| Bibb|Alabama|1007| 2051| 48| 2003|
2021-01-10| Blount|Alabama|1009| 5066| 77| 4989|
2021-01-10| Bullock|Alabama|1011| 953| 28| 925|
2021-01-10| Butler|Alabama|1013| 1632| 57| 1575|
2021-01-10| Calhoun|Alabama|1015|10497| 178|10319|
2021-01-10|Chambers|Alabama|1017| 2639| 66| 2573|
2021-01-10|Cherokee|Alabama|1019| 1537| 24| 1513|
2021-01-10| Chilton|Alabama|1021| 3253| 54| 3199|
2021-01-10| Choctaw|Alabama|1023| 501| 22| 479|
2021-01-10| Clarke|Alabama|1025| 2633| 26| 2607|
2021-01-10| Clay|Alabama|1027| 1230| 35| 1195|
2021-01-10|Cleburne|Alabama|1029| 1118| 18| 1100|
2021-01-10| Coffee|Alabama|1031| 4145| 43| 4102|
2021-01-10| Colbert|Alabama|1033| 5081| 61| 5020|
2021-01-10| Conecuh|Alabama|1035| 907| 18| 889|
2021-01-10| Coosa|Alabama|1037| 579| 9| 570|
+----------+--------+-------+----+-----+------+-----+
only showing top 20 rows

##### Get counties with the most cases

In [0]:
covid_df.select("county", "cases").groupBy("county").agg(max("cases").alias("max_cases")).sort(col("max_cases").desc()).limit(5).show()

+-------------+---------+
 county|max_cases|
+-------------+---------+
 Los Angeles| 920560|
New York City| 487551|
 Cook| 417790|
 Maricopa| 382409|
 Miami-Dade| 326606|
+-------------+---------+

In [0]:
patients_df.show()
patients_df.printSchema()

+--------------------+----------+----------+-----------+---------+----------+------+-------+----------+------+-----------+-------+--------+---------------+------+--------------------+--------------------+---------------+-------------+----+
 Id| BIRTHDATE| DEATHDATE| SSN| DRIVERS| PASSPORT|PREFIX| FIRST| LAST|SUFFIX| MAIDEN|MARITAL| RACE| ETHNICITY|GENDER| BIRTHPLACE| ADDRESS| CITY| STATE| ZIP|
+--------------------+----------+----------+-----------+---------+----------+------+-------+----------+------+-----------+-------+--------+---------------+------+--------------------+--------------------+---------------+-------------+----+
a45a0f84-1d16-451...|1979-04-05| null|999-95-4871|S99995591| X718276X| Mr.|Gilbert| Medhurst| null| null| M| white| american| M|West Tisbury Mas...|696 Schmeler Stra...| Leominster|Massachusetts|1420|
cc3e87e4-6172-45c...|1979-04-01| null|999-56-1714|S99954952|X26053750X| Mr.| Corey| Hoeger| null| null| M| white| english| M|East Bridgewater ...|1080 Klein Highla...| Newton|Massachusetts|2458|
4edfaae9-eef0-430...|1948-01-09| null|999-65-8945|S99996643|X78731815X| Mrs.| Brook| Dooley| null|Greenfelder| M| white|french_canadian| F|Springfield Mass...|253 Gutkowski App...| Belmont|Massachusetts|2138|
7ff0403d-6cc4-48a...|1956-01-12| null|999-67-6216|S99945217|X30828362X| Mr.|Norbert| Corkery| null| null| M| white| german| M|Pembroke Massach...| 520 White Bay|Barnstable Town|Massachusetts|2601|
2f905f97-cf1f-4f0...|1998-05-30| null|999-28-1698|S99990095| X1436582X| Ms.| Maisie|McCullough| null| null| null| white|french_canadian| F|Worcester Massac...| 118 Marks Port| Lynn|Massachusetts|1901|
b9b551cf-adf6-4f2...|1970-09-05|2014-03-15|999-19-5833|S99996196|X26794549X| Mr.| Monte|Macejkovic| null| null| M| black| dominican| M|Brookline Massac...|995 Heller Rest A...| Springfield|Massachusetts|1013|
e747f9b3-480b-4ce...|1994-07-07| null|999-75-6843|S99958855|X20317878X| Ms.|Lucinda| Bayer| null| null| null| white| irish| F|Melrose Massachu...|338 Lebsack Glen ...| Ware|Massachusetts|1082|
61bb8fc6-b6ce-472...|1990-09-28| null|999-60-2463|S99916176|X57174560X| Mrs.| Carla| Badillo| null| Ontiveros| M|hispanic| mexican| F|Mexico City Mexi...|952 Farrell Boule...| Peabody|Massachusetts|1940|
13e5d9d1-9756-4d5...|2011-09-20| null|999-79-5029| null| null| null|Analisa|Morissette| null| null| null| white| irish| F|Templeton Massac...| 949 Turner Lane| Middleborough|Massachusetts|2346|
3919e818-9080-4c1...|1998-02-09| null|999-28-8651|S99915632|X72780201X| Mr.| Quincy| Lebsack| null| null| null| white| french| M|Townsend Massach...| 452 Brakus Gardens| Chelsea|Massachusetts|2149|
fdd3cddc-7b4a-468...|1988-10-01| null|999-11-6581|S99953793| X5626852X| Mr.| Errol| Tillman| null| null| M| white| polish| M|Tewksbury Massac...|783 Walsh Ranch A...| Somerville|Massachusetts|2138|
26aaef22-2840-4ce...|1986-02-26| null|999-58-6994|S99952390| X4942798X| Mr.| Jerald| Waters| null| null| S| white| swedish| M|Swansea Massachu...|413 Boyle Station...| Chelmsford|Massachusetts|1824|
d6b2eece-060d-41d...|2015-03-01| null|999-12-1401| null| null| null| Kip| Mitchell| null| null| null| white| french| M|Bellingham Massa...| 606 Strosin Park| Boston|Massachusetts|2108|
e958b972-c33c-455...|1975-04-24| null|999-17-4292|S99947722|X55742300X| Mr.| Pablo| Rivero| null| null| M| black| west_indian| M|Santiago de los C...|555 Langworth Pat...| Ayer|Massachusetts|1432|
9bbe5f47-dbac-4ff...|1970-09-05| null|999-20-6466|S99963920|X21871050X| Mr.| Judson| Franecki| null| null| S| black| dominican| M|Hanson Massachus...| 759 Hickle Fort| Springfield|Massachusetts|1013|
4b575928-56c0-4cf...|1949-01-03| null|999-41-2337|S99933266|X32569297X| Mr.| Stuart| Fadel| null| null| S| asian| asian_indian| M|Springfield Mass...|813 Ebert Union A...| Boston|Massachusetts|2108|
079a81dd-9e6b-4ce...|1991-02-06| null|999-68-4829|S99926906|X51211096X| Mr.| Bryon| Daugherty| null| null| M| native|american_indian| M|Quincy Massachus...|496 Kertzmann Cam...| Winthr

##### Make patients DF easier to use

In [0]:
new_patients_df = patients_df.drop("SSN", "DRIVERS","PASSPORT")
new_patients_df.show()

+--------------------+----------+----------+------+-------+----------+------+-----------+-------+--------+---------------+------+--------------------+--------------------+---------------+-------------+----+
 Id| BIRTHDATE| DEATHDATE|PREFIX| FIRST| LAST|SUFFIX| MAIDEN|MARITAL| RACE| ETHNICITY|GENDER| BIRTHPLACE| ADDRESS| CITY| STATE| ZIP|
+--------------------+----------+----------+------+-------+----------+------+-----------+-------+--------+---------------+------+--------------------+--------------------+---------------+-------------+----+
a45a0f84-1d16-451...|1979-04-05| null| Mr.|Gilbert| Medhurst| null| null| M| white| american| M|West Tisbury Mas...|696 Schmeler Stra...| Leominster|Massachusetts|1420|
cc3e87e4-6172-45c...|1979-04-01| null| Mr.| Corey| Hoeger| null| null| M| white| english| M|East Bridgewater ...|1080 Klein Highla...| Newton|Massachusetts|2458|
4edfaae9-eef0-430...|1948-01-09| null| Mrs.| Brook| Dooley| null|Greenfelder| M| white|french_canadian| F|Springfield Mass...|253 Gutkowski App...| Belmont|Massachusetts|2138|
7ff0403d-6cc4-48a...|1956-01-12| null| Mr.|Norbert| Corkery| null| null| M| white| german| M|Pembroke Massach...| 520 White Bay|Barnstable Town|Massachusetts|2601|
2f905f97-cf1f-4f0...|1998-05-30| null| Ms.| Maisie|McCullough| null| null| null| white|french_canadian| F|Worcester Massac...| 118 Marks Port| Lynn|Massachusetts|1901|
b9b551cf-adf6-4f2...|1970-09-05|2014-03-15| Mr.| Monte|Macejkovic| null| null| M| black| dominican| M|Brookline Massac...|995 Heller Rest A...| Springfield|Massachusetts|1013|
e747f9b3-480b-4ce...|1994-07-07| null| Ms.|Lucinda| Bayer| null| null| null| white| irish| F|Melrose Massachu...|338 Lebsack Glen ...| Ware|Massachusetts|1082|
61bb8fc6-b6ce-472...|1990-09-28| null| Mrs.| Carla| Badillo| null| Ontiveros| M|hispanic| mexican| F|Mexico City Mexi...|952 Farrell Boule...| Peabody|Massachusetts|1940|
13e5d9d1-9756-4d5...|2011-09-20| null| null|Analisa|Morissette| null| null| null| white| irish| F|Templeton Massac...| 949 Turner Lane| Middleborough|Massachusetts|2346|
3919e818-9080-4c1...|1998-02-09| null| Mr.| Quincy| Lebsack| null| null| null| white| french| M|Townsend Massach...| 452 Brakus Gardens| Chelsea|Massachusetts|2149|
fdd3cddc-7b4a-468...|1988-10-01| null| Mr.| Errol| Tillman| null| null| M| white| polish| M|Tewksbury Massac...|783 Walsh Ranch A...| Somerville|Massachusetts|2138|
26aaef22-2840-4ce...|1986-02-26| null| Mr.| Jerald| Waters| null| null| S| white| swedish| M|Swansea Massachu...|413 Boyle Station...| Chelmsford|Massachusetts|1824|
d6b2eece-060d-41d...|2015-03-01| null| null| Kip| Mitchell| null| null| null| white| french| M|Bellingham Massa...| 606 Strosin Park| Boston|Massachusetts|2108|
e958b972-c33c-455...|1975-04-24| null| Mr.| Pablo| Rivero| null| null| M| black| west_indian| M|Santiago de los C...|555 Langworth Pat...| Ayer|Massachusetts|1432|
9bbe5f47-dbac-4ff...|1970-09-05| null| Mr.| Judson| Franecki| null| null| S| black| dominican| M|Hanson Massachus...| 759 Hickle Fort| Springfield|Massachusetts|1013|
4b575928-56c0-4cf...|1949-01-03| null| Mr.| Stuart| Fadel| null| null| S| asian| asian_indian| M|Springfield Mass...|813 Ebert Union A...| Boston|Massachusetts|2108|
079a81dd-9e6b-4ce...|1991-02-06| null| Mr.| Bryon| Daugherty| null| null| M| native|american_indian| M|Quincy Massachus...|496 Kertzmann Cam...| Winthrop Town|Massachusetts|2128|
6d19b7b0-1d93-48a...|1961-04-12| null| Mrs.| Randi| Mraz| null| Mayert| M| white| scottish| F|Auburn Massachus...|484 Ullrich Burg ...| New Bedford|Massachusetts|2740|
f4ed4f7f-fdce-420...|2014-12-18| null| null| Jayson| Collier| null| null| null| white| italian| M|Dennis Massachus...|1022 Stanton Ligh...| Boston|Massachusetts|2108|
80f3ad76-d17c-46a...|2004-06-08| null| null| Cinda| Herman| null| null| null|hispanic| south_american| F|Santiago Santiag...| 512 Hickle Mall| Lawrence|Massachusetts|1840|
+--------------------+----------+----------+------+-------+----------+------+-----------+-------+

In [0]:
new_patients_df.printSchema()

root
-- Id: string (nullable = true)
-- BIRTHDATE: string (nullable = true)
-- DEATHDATE: string (nullable = true)
-- PREFIX: string (nullable = true)
-- FIRST: string (nullable = true)
-- LAST: string (nullable = true)
-- SUFFIX: string (nullable = true)
-- MAIDEN: string (nullable = true)
-- MARITAL: string (nullable = true)
-- RACE: string (nullable = true)
-- ETHNICITY: string (nullable = true)
-- GENDER: string (nullable = true)
-- BIRTHPLACE: string (nullable = true)
-- ADDRESS: string (nullable = true)
-- CITY: string (nullable = true)
-- STATE: string (nullable = true)
-- ZIP: integer (nullable = true)

###### Patients with the longest last names

In [0]:
patients_name_DF =new_patients_df.select("LAST").distinct().selectExpr("*", "length(LAST) as length")
patients_name_DF.orderBy(col('length').desc()).limit(10).show()


+-------------+------+
 LAST|length|
+-------------+------+
Runolfsdottir| 13|
 Christiansen| 12|
 Pfannerstill| 12|
 Carrasquillo| 12|
 Schamberger| 11|
 Cruickshank| 11|
 Ballesteros| 11|
 Satterfield| 11|
 Oberbrunner| 11|
 Covarrubias| 11|
+-------------+------+

##### Number of patients from each City

In [0]:
new_patients_df.select("Id", "City").groupBy("City").agg(count("Id").alias("Number of patients")).show()


+------------------+------------------+
 City|Number of patients|
+------------------+------------------+
 Nahant| 6|
 Worcester| 305|
 Hanover| 24|
 Middlefield| 1|
 Springfield| 252|
 Auburn| 24|
 Georgetown| 13|
 Savoy| 2|
 Wrentham| 21|
 Granville| 1|
 Waltham| 110|
 Fitchburg| 76|
 Belchertown| 15|
 Tisbury| 6|
 Sterling| 24|
North Attleborough| 72|
 Brookfield| 2|
 Conway| 4|
 Bolton| 12|
 Winchendon| 20|
+------------------+------------------+
only showing top 20 rows

## Using SQL in Spark SQL

In [0]:
infoDF = spark.sql("select current_date() as today , 1 + 100 as value")
infoDF.show()


+----------+-----+
 today|value|
+----------+-----+
2021-01-26| 101|
+----------+-----+

In [0]:
covid_df.createOrReplaceTempView("covid")

In [0]:
spark.sql("SELECT * FROM covid").show()

+----------+-----------+----------+-----+-----+------+
 Date| county| state| fips|cases|deaths|
+----------+-----------+----------+-----+-----+------+
2020-01-21| Snohomish|Washington|53061| 1| 0|
2020-01-22| Snohomish|Washington|53061| 1| 0|
2020-01-23| Snohomish|Washington|53061| 1| 0|
2020-01-24| Cook| Illinois|17031| 1| 0|
2020-01-24| Snohomish|Washington|53061| 1| 0|
2020-01-25| Orange|California| 6059| 1| 0|
2020-01-25| Cook| Illinois|17031| 1| 0|
2020-01-25| Snohomish|Washington|53061| 1| 0|
2020-01-26| Maricopa| Arizona| 4013| 1| 0|
2020-01-26|Los Angeles|California| 6037| 1| 0|
2020-01-26| Orange|California| 6059| 1| 0|
2020-01-26| Cook| Illinois|17031| 1| 0|
2020-01-26| Snohomish|Washington|53061| 1| 0|
2020-01-27| Maricopa| Arizona| 4013| 1| 0|
2020-01-27|Los Angeles|California| 6037| 1| 0|
2020-01-27| Orange|California| 6059| 1| 0|
2020-01-27| Cook| Illinois|17031| 1| 0|
2020-01-27| Snohomish|Washington|53061| 1| 0|
2020-01-28| Maricopa| Arizona| 4013| 1| 0|
2020-01-28|Los Angeles|California| 6037| 1| 0|
+----------+-----------+----------+-----+-----+------+
only showing top 20 rows

In [0]:
spark.sql("SELECT max(cases) AS max_cases, max(deaths) AS max_deaths, county\
          FROM covid\
          GROUP BY county\
          ORDER BY max_cases DESC\
          LIMIT 10"
         ).show()




+---------+----------+--------------+
max_cases|max_deaths| county|
+---------+----------+--------------+
 920560| 12250| Los Angeles|
 487551| 25562| New York City|
 417790| 8760| Cook|
 382409| 5818| Maricopa|
 326606| 4413| Miami-Dade|
 262525| 3588| Harris|
 228857| 1531|San Bernardino|
 217219| 2239| Dallas|
 216076| 2218| Riverside|
 199401| 2091| Orange|
+---------+----------+--------------+